In [3]:
import torch
from PIL import Image
from transformers import AutoModel
from transformers import AutoTokenizer
from huggingface_hub import login
from PIL import Image
import h5py

import numpy as np
import os
login("Your hugging face code")
import h5py
import os
import pickle
import yaml
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import torch
from transformers import AutoModel
from typing import Union, Dict, List, Tuple, Optional,Any

/Users/zz/Desktop/virtualenvs/pathology_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch
import numpy as np
import h5py
from PIL import Image
from pathlib import Path
import pandas as pd
from transformers import AutoModel

root_dir = Path("/Users/zz/Desktop/reser/ruiming/pathology/glomerulus/Glom_Patches_nopatches")
output_dir = root_dir / "h5"
output_dir.mkdir(parents=True, exist_ok=True)

patch_size = 512
patch_size_lv0 = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
conch, eval_transform = titan.return_conch()
conch = conch.to(device).eval()

records = []

for class_folder in root_dir.iterdir():
    if class_folder.is_dir() and class_folder.name != "h5":
        label = class_folder.name
        for png_file in class_folder.glob("*.png"):
            try:
                img = Image.open(png_file).convert("RGB")
                width, height = img.size
                features, coords = [], []

                for y in range(0, height - patch_size + 1, patch_size):
                    for x in range(0, width - patch_size + 1, patch_size):
                        patch = img.crop((x, y, x + patch_size, y + patch_size))
                        patch_tensor = eval_transform(patch).unsqueeze(0).to(device)
                        with torch.no_grad():
                            feat = conch(patch_tensor)
                        features.append(feat.squeeze(0).cpu().numpy())
                        coords.append([x, y])

                if not features:
                    print(f" skip empty image：{png_file}")
                    continue

                features = np.stack(features)
                coords = np.stack(coords)

                output_h5_path = output_dir / f"{png_file.stem}.h5"
                with h5py.File(output_h5_path, "w") as f:
                    f.create_dataset("features", data=features)
                    dset = f.create_dataset("coords", data=coords)
                    dset.attrs["patch_size_level0"] = patch_size_lv0

                print(f"✅ H5 saved：{output_h5_path.name}")
                records.append({
                    "h5_filename": output_h5_path.name,
                    "label": label
                })
            except Exception as e:
                print(f" failed: {png_file}，error：{e}")
df = pd.DataFrame(records)
df.to_csv(output_dir / "labels.csv", index=False)
print(f"\n save to: {output_dir / 'labels.csv'}")


✅ H5 saved：patch_1_35817_10209_image.h5
✅ H5 saved：patch_2_5304_7338_image.h5
✅ H5 saved：patch_1_10237_19975_image.h5
✅ H5 saved：patch_2_17277_9260_image.h5
✅ H5 saved：patch_2_12717_5064_image.h5
✅ H5 saved：patch_2_27791_8949_image.h5
✅ H5 saved：patch_2_23032_8645_image.h5
✅ H5 saved：patch_1_6345_20987_image.h5
✅ H5 saved：patch_2_6961_7535_image.h5
✅ H5 saved：patch_2_2251_12393_image.h5
✅ H5 saved：patch_2_7004_7025_image.h5
✅ H5 saved：patch_2_4907_6823_image.h5
✅ H5 saved：patch_2_11988_6899_image.h5
✅ H5 saved：patch_2_6660_6840_image.h5
✅ H5 saved：patch_2_5643_7008_image.h5
✅ H5 saved：patch_1_42726_14018_image.h5
✅ H5 saved：patch_2_4474_8136_image.h5
✅ H5 saved：patch_2_12614_6361_image.h5
✅ H5 saved：patch_1_19364_24550_image.h5
✅ H5 saved：patch_1_23630_33133_image.h5
✅ H5 saved：patch_2_1741_14575_image.h5
✅ H5 saved：patch_2_4532_10607_image.h5
✅ H5 saved：patch_1_4066_25673_image.h5
✅ H5 saved：patch_2_4961_8253_image.h5
✅ H5 saved：patch_2_25461_5694_image.h5
✅ H5 saved：patch_1_26409_325